## DPP(物品多样性算法)

注意1：item emb的维度 N 必须大于召回的物品数量k，否则 det（S）== 0, 算法失效。

注意2：物品向量需要先标准化norm(emb) = 1, 否则会在数值算法推导过程中出现偏差（参考王树森的note）。


存在的问题：
    比如mmr例子中的emb dim = 2 三个物品，我们选择两个k=2
    由于k <= emb_dim, 但是dpp的物理意义在二维情况下是两个向量围起来的最大面积，
    最不相似的 item3 与 item1构成的面积，反而没有最相似的 item2 与 item1的面积大。
    从而并没有达到多样性的需求。

    总结：
        dpp使用矩阵的det作为衡量标准，但是A = V @ V.T, 导致detV**2永远为正数。
        是不是说dpp选择的物品是相似度并不是存粹向量的相似度，
        只是数学意义上的可表示相似度，负相似物品也可以用正相似物品表示。
        


In [1]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys 
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [2]:
import numpy as np
import random
from sgd_rec_sys.reorder import dpp, Item

In [3]:
emb = np.random.randn(1,4)
print(emb @ emb.T)

emb_norm = emb/ np.linalg.norm(emb)
print(emb_norm @ emb_norm.T)

[[4.47099885]]
[[1.]]


### 0、物品队列准备

In [4]:
emb_N = 10
np.random.seed(1) # 随机emb可复现

# 准备四个物品，选其中的两个
a1 = Item(id=1)
a1.set_reward(0.6)
a1.set_emb(np.random.randn(emb_N))

a2 = Item(id=2)
a2.set_reward(0.5)
a2.set_emb(np.random.randn(emb_N))

a3 = Item(id=3)
a3.set_reward(0.5)
a3.set_emb(np.random.randn(emb_N))

a4 = Item(id=4)
a4.set_reward(0.5)
a4.set_emb(np.random.randn(emb_N))

items= [a1,a2,a3, a4]
index = {item.id: item for item in items}

### 1、 numpy det cholesky相关函数测试

In [5]:
aid = [1,2]
bid = [1,3]

def get_matrix(ids):
    X = np.array([index[id].get_norm_emb() for id in ids])
    X.reshape(len(ids), -1)
    return X
    
A = get_matrix(aid)
B = get_matrix(bid)
print(A, A.shape)

print(A, A.shape, np.linalg.det(A @ A.T))
print(B, B.shape, np.linalg.det(B @ B.T))

[[ 0.42989618 -0.1619063  -0.13978494 -0.28396985  0.22903715 -0.60912089
   0.46177859 -0.20145958  0.08443629 -0.06599789]
 [ 0.45623187 -0.64284026 -0.10060612 -0.11983919  0.35377809 -0.34320684
  -0.05380399 -0.27392437  0.01317225  0.18185995]] (2, 10)
[[ 0.42989618 -0.1619063  -0.13978494 -0.28396985  0.22903715 -0.60912089
   0.46177859 -0.20145958  0.08443629 -0.06599789]
 [ 0.45623187 -0.64284026 -0.10060612 -0.11983919  0.35377809 -0.34320684
  -0.05380399 -0.27392437  0.01317225  0.18185995]] (2, 10) 0.5672492314307513
[[ 0.42989618 -0.1619063  -0.13978494 -0.28396985  0.22903715 -0.60912089
   0.46177859 -0.20145958  0.08443629 -0.06599789]
 [-0.44526454  0.46310739  0.36474594  0.20328822  0.36444868 -0.27660773
  -0.04971625 -0.37857322 -0.10837633  0.21455966]] (2, 10) 0.9913224280047033


### 1.1、验证cholesky分解的正确性

In [6]:
from numpy.linalg import  det, cholesky

AA = A @ A.T
BB = B @ B.T

L = cholesky(AA)
print(AA)
print( L @ L.T)
print(det(AA), det(L)**2)

[[1.         0.65783795]
 [0.65783795 1.        ]]
[[1.         0.65783795]
 [0.65783795 1.        ]]
0.5672492314307513 0.5672492314307515


### 1.2、验证dpp递推公式计算det的正确性

In [7]:
# step t：真实V_s  
V = get_matrix([1,2])
As = V @ V.T
print("Asi_true", As)
print("L_asi", cholesky(As))

print('---' * 4)

# step t+1:真实V_si
Vsi =  get_matrix([1,2, 3])
Asi = Vsi @ Vsi.T
print("Asi_true", Asi)
print("L_asi", cholesky(Asi))

Asi_true [[1.         0.65783795]
 [0.65783795 1.        ]]
L_asi [[1.         0.        ]
 [0.65783795 0.7531595 ]]
------------
Asi_true [[ 1.          0.65783795 -0.09315349]
 [ 0.65783795  1.         -0.19407028]
 [-0.09315349 -0.19407028  1.        ]]
L_asi [[ 1.          0.          0.        ]
 [ 0.65783795  0.7531595   0.        ]
 [-0.09315349 -0.17631111  0.97991674]]


In [8]:
# 由L_s 递推到L_si 的中间步骤

L_As = cholesky(As)
# 由As递推Asi，基于cholesky 分解， As = L @ L.T
ai = np.array([index[3].get_norm_emb() @ index[sid].get_norm_emb() for sid in [1, 2]])
ai = ai.reshape(-1, 1)
print('ai', ai)
print("ai.shape", ai.shape)
print("L and L_inv's shape", L_As.shape, np.linalg.inv(L_As).shape)

ci = np.linalg.inv(L_As) @ ai
print('ci', ci)
print(ci.T@ci)
print("ci.shape", ci.shape)

di = np.sqrt(1-ci.T@ci)
print('di', di)
print("new det:", (det(L_As) * di)**2)

# 计算当前候选item的det值
next_det = (det(L_As) * di)**2
print("det_L_As:{}, di:{}".format(det(L_As), di))

print("keep same:", det(Asi), next_det)
assert abs(det(Asi)- next_det) < 1e-7
print("true det == 递归det")   

ai [[-0.09315349]
 [-0.19407028]]
ai.shape (2, 1)
L and L_inv's shape (2, 2) (2, 2)
ci [[-0.09315349]
 [-0.17631111]]
[[0.03976318]]
ci.shape (2, 1)
di [[0.97991674]]
new det: [[0.5446936]]
det_L_As:0.7531594993298242, di:[[0.97991674]]
keep same: 0.5446935990443732 [[0.5446936]]
true det == 递归det


In [9]:
# 递推更新L。

# 右边补上0
m,n = L_As.shape
L_As_pad = np.zeros((m,n+1))  
L_As_pad[:,:n] = L_As
print("L_As", L_As, L_As.shape)
print("L_As_pad", L_As_pad, L_As_pad.shape)

# 下面补上一行（ci，di）
print("ci, di", ci,ci.shape, di )
cidi = np.concatenate((ci.T, np.array(di)), axis=1)
print("cidi", cidi, cidi.shape)

next_L = np.concatenate((L_As_pad, cidi), axis=0)  
print("next_L", next_L)
print("True_next_L", cholesky(Asi))

L_As [[1.         0.        ]
 [0.65783795 0.7531595 ]] (2, 2)
L_As_pad [[1.         0.         0.        ]
 [0.65783795 0.7531595  0.        ]] (2, 3)
ci, di [[-0.09315349]
 [-0.17631111]] (2, 1) [[0.97991674]]
cidi [[-0.09315349 -0.17631111  0.97991674]] (1, 3)
next_L [[ 1.          0.          0.        ]
 [ 0.65783795  0.7531595   0.        ]
 [-0.09315349 -0.17631111  0.97991674]]
True_next_L [[ 1.          0.          0.        ]
 [ 0.65783795  0.7531595   0.        ]
 [-0.09315349 -0.17631111  0.97991674]]


### 2、test：DPP算法

In [10]:
# 四个物品中选出三个（兼顾多样性&reward）
theta = 0.5
k = 3

ids = dpp(items, k, theta, w=0)
print(ids) # [1, 3, 4]


rid: 2 ***************
rid:2, score:[[-0.03347826]]

rid: 3 ***************
rid:3, score:[[0.24564228]]

rid: 4 ***************
rid:4, score:[[0.23034454]]

rid: 2 ***************
rid:2, score:[[-0.05376592]]

rid: 4 ***************
rid:4, score:[[0.19362072]]
[1, 3, 4]


###  3、test：DPP带窗口

In [11]:
# 四个物品中选出三个（兼顾多样性&reward）
theta = 0.5
k = 3

ids = dpp(items, k, theta, w=1)
print(ids) # [1, 3, 2]


rid: 2 ***************
rid:2, score:[[-0.03347826]]

rid: 3 ***************
rid:3, score:[[0.24564228]]

rid: 4 ***************
rid:4, score:[[0.23034454]]

rid: 2 ***************
rid:2, score:[[0.23080457]]

rid: 4 ***************
rid:4, score:[[0.22375504]]
[1, 3, 2]


In [12]:
# 简单分析一下带窗口的情况，由于item2、3、4 reward设置的都一样，
# 每轮选item只需要与上一个向量围成的面积最大。

def calc_det(ids):
    V = get_matrix(ids)
    As =  V @ V.T
    return det(As)

# 根据最大的det选择下一个物品

# 第一轮：
det_score = [calc_det([1]+[id]) for id in [2, 3, 4] ]
print(det_score) # [0.5672492314307513, 0.9913224280047033, 0.9614517294073714] 所以第一轮选择item3

# 不带窗口第二轮：
det_score = [calc_det([1, 3]+[id]) for id in [2, 4]]
print(det_score) # [0.5446935990443731, 0.8933663285472728] 不带窗口第二轮选择item4

# 带窗口第二轮:
det_score = [calc_det([3]+[id]) for id in [2, 4]]
print(det_score) # [0.9623367251039103, 0.948863883590221]  带窗口第二轮选择item2


[0.5672492314307513, 0.9913224280047033, 0.9614517294073714]
[0.5446935990443731, 0.8933663285472728]
[0.9623367251039103, 0.948863883590221]
